In [ ]:
import pandas as pd
import numpy as np
from pulp import *


In [1]:
# Load files

df = pd.read_excel("Downloads/fin.xlsx", sheet_name="Sheet1")
cf = pd.read_excel("Downloads/fin.xlsx", sheet_name="Sheet2")

In [30]:
# Capacity expansion model

cf = cf.rename(columns={"Solar_unit": "Solar_CF", "Wind_unit": "Wind_CF"})
df_model = df.copy()
df_model["Solar_CF"] = cf["Solar_CF"]
df_model["Wind_CF"] = cf["Wind_CF"]
n = len(df_model)

# Model setup
model = LpProblem("Simple_Capacity_Expansion", LpMinimize)

# Decision variables
solar_cap = LpVariable("Solar_Capacity_MW", lowBound=0)
wind_cap = LpVariable("Wind_Capacity_MW", lowBound=0)
bess_energy = LpVariable("BESS_Energy_MWh", lowBound=0)
bess_power = LpVariable("BESS_Power_MW", lowBound=0)

# Parameters
solar_capex = 4.00
wind_capex = 7.00
bess_power_capex = 1.33
bess_energy_capex = 3.00
eta = 0.9

# Total energy demand
total_energy = df_model["Demand"].sum() * 0.25  # MWh over 15-min intervals

# Total energy from each source
solar_energy = lpSum([solar_cap * df_model.loc[t, "Solar_CF"] * 0.25 for t in range(n)])
wind_energy = lpSum([wind_cap * df_model.loc[t, "Wind_CF"] * 0.25 for t in range(n)])
bess_energy_supplied = eta * bess_energy

# Objective: Minimize total capital cost
model += (
    solar_capex * solar_cap +
    wind_capex * wind_cap +
    bess_power_capex * bess_power +
    bess_energy_capex * bess_energy
)

# Constraints
model += solar_energy + wind_energy + bess_energy_supplied >= total_energy
model += solar_energy >= 0.5 * total_energy
model += wind_energy >= 0.3 * total_energy
model += wind_energy <= 0.4 * total_energy

# Enforce basic minimums
peak_demand = df_model["Demand"].max()
model += bess_power >= 0.1 * peak_demand
model += bess_energy >= 0.2 * peak_demand

# Solve
model.solve()

# Results
print("\n✅ SIMPLE CAPACITY EXPANSION RESULTS:")
print("Solar Capacity (MW):", round(value(solar_cap), 2))
print("Wind Capacity (MW):", round(value(wind_cap), 2))
print("BESS Power (MW):", round(value(bess_power), 2))
print("BESS Energy (MWh):", round(value(bess_energy), 2))
print("Total Capex (Rs Cr):", round(value(model.objective), 2))



✅ SIMPLE CAPACITY EXPANSION RESULTS:
Solar Capacity (MW): 1081.35
Wind Capacity (MW): 361.86
BESS Power (MW): 126.0
BESS Energy (MWh): 252.0
Total Capex (Rs Cr): 7781.95


In [33]:
# Economic dispatch
# Merge solar/wind capacity factors into demand data
cf = cf.rename(columns={"Solar_unit": "Solar_CF", "Wind_unit": "Wind_CF"})
df_model = df.copy()
df_model["Solar_CF"] = cf["Solar_CF"]
df_model["Wind_CF"] = cf["Wind_CF"]

# Installed capacities (MW) and efficiencies
solar_cap = 1081.35
wind_cap = 361.86
bess_power = 126
bess_energy = 252
eta_c = 0.92
eta_d = 0.92

# Initialize storage
soc = 0
soc_series = []
solar_gen_list = []
wind_gen_list = []
bess_discharge_list = []
bess_charge_list = []
shortfall_list = []

# Time loop
for t in range(len(df_model)):
    demand = df_model.loc[t, "Demand"]
    solar_avail = solar_cap * df_model.loc[t, "Solar_CF"]
    wind_avail = wind_cap * df_model.loc[t, "Wind_CF"]

    # Solar dispatch
    solar_dispatch = min(solar_avail, demand)

    # Wind dispatch
    wind_dispatch = min(wind_avail, demand - solar_dispatch)

    # Battery discharge
    required_after_re = demand - solar_dispatch - wind_dispatch
    bess_discharge = min(soc * eta_d, bess_power, required_after_re)
    soc -= bess_discharge / eta_d

    # Excess RE for charging
    solar_left = solar_avail - solar_dispatch
    wind_left = wind_avail - wind_dispatch
    re_left = solar_left + wind_left
    bess_charge = min(re_left, bess_power, (bess_energy - soc) / eta_c)
    soc += bess_charge * eta_c

    # Attribute charge to solar/wind
    if re_left > 0:
        if solar_left >= bess_charge:
            solar_dispatch += bess_charge
        elif solar_left + wind_left >= bess_charge:
            solar_dispatch += solar_left
            wind_dispatch += bess_charge - solar_left
        else:
            solar_dispatch += solar_left
            wind_dispatch += wind_left

    # Shortfall (if any)
    shortfall = demand - (solar_dispatch + wind_dispatch + bess_discharge)

    # Record timestep data
    soc_series.append(soc)
    solar_gen_list.append(solar_dispatch)
    wind_gen_list.append(wind_dispatch)
    bess_discharge_list.append(bess_discharge)
    bess_charge_list.append(bess_charge)
    shortfall_list.append(shortfall)

# Combine into DataFrame
dispatch_df = df_model.copy()
dispatch_df["Solar_Gen"] = solar_gen_list
dispatch_df["Wind_Gen"] = wind_gen_list
dispatch_df["BESS_Discharge"] = bess_discharge_list
dispatch_df["BESS_Charge"] = bess_charge_list
dispatch_df["SOC"] = soc_series
dispatch_df["Shortfall"] = shortfall_list

# Save to Excel
dispatch_df.to_excel("dispatch_output_s.xlsx", index=False)
print("✅ Dispatch saved to 'dispatch_output_s.xlsx'")



✅ Dispatch saved to 'dispatch_output_s.xlsx'
